# AQLM 加性量化语言模型

尝试在 [Google Colab](https://colab.research.google.com/drive/1-xZmBRXT5Fm3Ghn4Mwa2KRypORXb855X?usp=sharing) 上使用 AQLM！

加性量化语言模型（AQLM）是一种大型语言模型压缩方法。它通过量化多个权重并利用它们之间的相互依赖关系来实现压缩。AQLM 将 8-16 个权重表示为多个向量代码的总和。

AQLM 的推理支持由 `aqlm` 库实现。确保安装该库以运行模型（注意：aqlm 只支持 python>=3.10）：


In [ ]:
pip install aqlm[gpu,cpu]


该库提供了高效的 GPU 和 CPU 推理和训练内核。

如何自己量化模型以及所有相关代码的说明可以在相应的 [GitHub 仓库](https://github.com/Vahe1994/AQLM) 中找到。要运行 AQLM 模型，只需加载已使用 AQLM 量化的模型：


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 加载量化的模型
quantized_model = AutoModelForCausalLM.from_pretrained(
    "ISTA-DASLab/Mixtral-8x7b-AQLM-2Bit-1x16-hf",
    torch_dtype="auto", 
    device_map="auto"
)

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained("ISTA-DASLab/Mixtral-8x7b-AQLM-2Bit-1x16-hf")


## PEFT

从版本 `aqlm 1.0.2` 开始，AQLM 支持参数高效的微调（PEFT），具体形式为集成到 [PEFT](https://huggingface.co/blog/peft) 库中的 [LoRA](https://huggingface.co/docs/peft/package_reference/lora)。

## AQLM 配置

AQLM 量化设置主要取决于使用的码本数量以及码本大小（以位为单位）。最常见的设置及其支持的推理内核如下表所示：

| 内核 | 码本数量 | 码本大小，位 | 表示法 | 准确度 | 加速比 | 快速 GPU 推理 | 快速 CPU 推理 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Triton | K   | N   | KxN | -  | 最高约 0.7 倍 | ✅   | ❌   |
| CUDA | 1   | 16  | 1x16 | 最佳 | 最高约 1.3 倍 | ✅   | ❌   |
| CUDA | 2   | 8   | 2x8 | 较好 | 最高约 3.0 倍 | ✅   | ❌   |
| Numba | K   | 8   | Kx8 | 良好 | 最高约 4.0 倍 | ❌   | ✅   |